## Instructions:
In the lesson materials, there is a file in the data folder called “water.xyz”. This is a very simple, standard file format that is often used to distribute molecular coordinates. The first line of the file is the number of atoms in the molecule, the second line is a title line (or may be blank), and the coordinates begin on the third line. The format of the coordinates is
Atom_Label  XCoor   YCoor   ZCoor

Write a code to read in the information from the xyz file and determine the bond lengths between all the atoms. There is a numpy function to take the square root, numpy.sqrt(). To raise a number to a power, use " ** ", as in " 3**2 = 9 ". 

### Anatomy of an xyz file:
There is an xyz file for water
Used to convey molecular structures
1st line is # of atoms
2nd line is comment line (name of molecule)
3rd line on is the symbol, x, y, z coordinate
there will be as many lines as there are atoms in the file

We should write a code to read in infomation into 

Distance formula from geometry 

### Extensions to homework
#### Ext. 1
Your initial project calculated the distance between every set of atoms. However, some of these atoms aren’t really bonded to each other. H1 and H2 are not bonded for example, and all of the distances between an atom and itself are zero. Use a distance cutoff of 1.5 angstroms to define a bond (that is, if the bond length is greater than 1.5 angstroms, consider the atoms not bonded). Modify your code to only print the atoms that are actually bonded to each other.

#### Ext. 2
Some of these are actually the same bond length; for example, O to H1 and H1 to O refer to the same bond length. Remove the duplicates from your list.

#### Ext. 3
Write a your output to a text file called bond_lengths.txt instead of just printing it to the screen.

https://molssi-education.github.io/python_scripting_cms/04-tabular_data/index.html

In [1]:
import os.path
import numpy

In [4]:
ls data

benzene.xyz*               outfiles/
buckminsterfullerene.xyz*  sapt.out*
distance_data_headers.csv* water.xyz*


In [5]:
file_path = os.path.join('data', 'water.xyz')
print(file_path)

data/water.xyz


In [10]:
# create file pointer to reference opened file to read in
outfile = open(file_path, "r")

# read each line of the file 
water_file = outfile.readlines()

# now that you have read the file, close it
# otherwise this can cause problems as the file it still accessible to memory
outfile.close()

In [70]:
# Alternative way to read in file
# Must skip header for this to work. Warning "Some errors were detected, 3 columns instead of 1"
# since row 1 is one digit, it thinks the array is one column; so genfromtxt needs an array with the same number of columns

xyz_file = numpy.genfromtxt(fname = file_path, skip_header = 2, dtype= 'unicode')
print(xyz_file)

[['O' '0.000000' '-0.007156' '0.965491']
 ['H1' '-0.000000' '0.001486' '-0.003471']
 ['H2' '0.000000' '0.931026' '1.207929']]


In [ ]:
# In solution approach, next they seperate the symbols and the coordinates as they are different data types. 
symbols = xyz_file[:, 0] # all rows, column 1 to get name
coordinates = xyz_file[:, 1:] # all rows, column 2 and on to get numbers
coordinates = coordinates.astype(numpy.float) # change data type from string to float

In [40]:
print(water_file)
print(len(water_file)) # prints 5, so 2 header lines and 3 atom coordinates, as expected

['3\n', 'Water xyz file\n', 'O        0.000000     -0.007156      0.965491\n', 'H1      -0.000000      0.001486     -0.003471\n', 'H2       0.000000      0.931026      1.207929\n']
5


In [57]:
water_data = coordinates[2:]
print(water_data)
type(water_data)
len(water_data)

['O        0.000000     -0.007156      0.965491'
 'H1      -0.000000      0.001486     -0.003471'
 'H2       0.000000      0.931026      1.207929']


3

In [56]:
# Calculate the bond length between 2 atoms (to test code before setting it up as a loop)
atom1 = water_data[0]
atom1_split = atom1.split()
atom1_name = atom1_split[0]
atom1_X = float(atom1_split[1])
atom1_Y = float(atom1_split[2])
atom1_Z = float(atom1_split[3])
print(atom1_name, atom1_X, atom1_Y, atom1_Z)

atom2 = water_data[1]
atom2_split = atom2.split()
atom2_name = atom2_split[0]
atom2_X = float(atom2_split[1])
atom2_Y = float(atom2_split[2])
atom2_Z = float(atom2_split[3])
print(atom2_name, atom2_X, atom2_Y, atom2_Z)

distance_sum = ((atom1_X - atom2_X)**2) + ((atom1_Y - atom2_Y)**2) + ((atom1_Z - atom2_Z)**2)
D = numpy.sqrt(distance_sum)
print(distance_sum)
print(D)

O 0.0 -0.007156 0.965491
H1 -0.0 0.001486 -0.003471
0.938962041608
0.9690005374652793


In [76]:
# set up the calculation as a for loop to test every possible pair
atom_names = []
atom_coordinates = []

# open and create a file name
datafile = open('H2Obondlengths.txt','w+')

for atom in water_data:
    atom_split = atom.split()
    atom_name = atom_split[0]
    atom_names.append(atom_name)
    x, y, z = float(atom_split[1]), float(atom_split[2]), float(atom_split[3])
    atom_coordinates.append([x,y,z])
    #print(atom_name, x, y, z)

    
# tricky thing about this nested for loop is that the loop counts over the same thing
# I did the range function (which students usually do), but could also use the enumberate function
# len(water_data) gives the number of columns; could also pull this number from the file or take the length of the symbols array
for i in range(len(water_data)):
    for j in range(len(water_data)):
        # this if statement prevents recipricol comparisons
        if i < j:
            atom1 = atom_coordinates[i]
            atom2 = atom_coordinates[j]
            distance_sum = ((atom1[0] - atom2[0])**2) + ((atom1[1] - atom2[1])**2) + ((atom1[2] - atom2[2])**2)
            D = numpy.sqrt(distance_sum)
            # covalent bonds are less than 1.5, don't include bonds greater than 1.5
            # looking for bonds > 0 will cut out the comparisons of an atom to itself
            if D > 0 and D <= 1.5:
                print(f'{atom_names[i]} to {atom_names[j]} bond distance: {D: .3f} angstroms.')
                datafile.write(f'{atom_names[i]} to {atom_names[j]} bond distance: {D: .3f} angstroms. \n')

datafile.close()    


O to H1 bond distance:  0.969 angstroms.
O to H2 bond distance:  0.969 angstroms.


## Defining functions in python

In [83]:
# basic syntax
# def function_name(its_parameters):
#     put code here indented
#     could be lots of lines
#     could include loops, conditional statements, etc.
#     return value_from_function

# design a function to measure bond length in angstroms
# forget about where the data is coming from in this specific example, make it general
# point out that the parameters named in the function title in () should be used within the function code
# point out that what ever they return has to be created within the function
# there is no output here, but you have to run the cell in jupyter notebook so that the function is in memory
def calculate_distance(atom1_coord, atom2_coord):
    """
    The triple quotes allows multi line strings and can be used anywhere. 
    This is a doc string. This is where you write documentation for how your function works. 
    There are standards for how to do this, depending on the library. MolSSI uses numpy standards.
    By putting the doc string here before your code, it will be printed when the help(your_function_name) function is called.
    Inputs: x, y, z coordinates (in that order) for atom1 and atom2.
    Returns the bond distance between those two atoms in angstroms.
    """
    x_distance = atom1_coord[0] - atom2_coord[0]
    y_distance = atom1_coord[1] - atom2_coord[1]
    z_distance = atom1_coord[2] - atom2_coord[2]
    bond_distance = numpy.sqrt(x_distance**2 + y_distance**2 + z_distance**2)
    return bond_distance



In [80]:
datafile = open('H2Obondlengths2.txt','w+')

for i in range(len(water_data)):
    for j in range(len(water_data)):
        # this if statement prevents recipricol comparisons
        if i < j:
            # Calling the function calculate_distance()
            D = calculate_distance(atom_coordinates[i], atom_coordinates[j])
            
            # covalent bonds are less than 1.5, don't include bonds greater than 1.5
            # looking for bonds > 0 will cut out the comparisons of an atom to itself
            if D > 0 and D <= 1.5:
                print(f'{atom_names[i]} to {atom_names[j]} bond distance: {D: .3f} angstroms.')
                datafile.write(f'{atom_names[i]} to {atom_names[j]} bond distance: {D: .3f} angstroms. \n')

datafile.close()    


O to H1 bond distance:  0.969 angstroms.
O to H2 bond distance:  0.969 angstroms.


In [82]:
# Best practice: document your function so that there is information when you use the help() function
help(calculate_distance)

Help on function calculate_distance in module __main__:

calculate_distance(atom1_coord, atom2_coord)
    The triple quotes allows multi line strings and can be used anywhere. 
    This is a doc string. This is where you write documentation for how your function works. 
    There are standards for how to do this, depending on the library. MolSSI uses numpy standards.
    Inputs: x, y, z coordinates (in that order) for atom1 and atom2.
    Returns the bond distance between those two atoms in angstroms.



In [86]:
# Write a function called bond_check. Check and see if a distance is between 0 and 1.5 angstroms. 
# Return True or False (need to tell students that True and False are boolean operators and mean something specific to python. Most be capitalized)

def bond_check(bond_distance):
    """
    Input: a distance between two atoms, in angstroms
    Evaluates whether the bond is between 0 and 1.5 angtroms
    Output: True if bond is between 0 and 1.5 angstroms; else, False
    """
    if bond_distance > 0 and bond_distance <= 1.5:
        return True
    else:
        return False
    




In [87]:
help(bond_check)

Help on function bond_check in module __main__:

bond_check(bond_distance)
    Input: a distance between two atoms, in angstroms
    Evaluates whether the bond is between 0 and 1.5 angtroms
    Output: True if bond is between 0 and 1.5 angstroms; else, False



In [88]:
# Testing your function:
# test a variety of values: positive, negative, decimal, int
bond_check(4) # False

False

In [89]:
bond_check(0) # False

False

In [90]:
bond_check(-1) # False

False

In [91]:
bond_check(1.2) # True

True

In [92]:
bond_check(0.8) # True

True

In [95]:
# Write a function called bond_check. Check and see if a distance is between 0 and 1.5 angstroms. 
# Return True or False (need to tell students that True and False are boolean operators and mean something specific to python. Most be capitalized)
# Make the function more flexible and let the user decide the bind length they want to check
# Declare default values and make the min and max 
# Note for students that required parameters must be listed first; then list optional
# don't need to type the name of the required parameters, but list them in order
# must declare the name of optional parameters
def bond_check(bond_distance, minimum_length = 0, maximum_length = 1.5):
    """
    Input: Three numeric parameters:
        1) a distance between two atoms, in angstroms (required parameter)
        2) minimum acceptable bond length (optional parameter; default is 0)
        3) maximum acceptable bond length (optional parameter; default is 1.5)
    Evaluates whether the bond is between min and max angtroms
    Output: True if bond is between 0 and 1.5 angstroms; else, False
    """
    if bond_distance > minimum_length and bond_distance <= maximum_length:
        return True
    else:
        return False
    




In [96]:
bond_check(0.8) # True

True

In [97]:
bond_check(1.6) # False

False

In [98]:
bond_check(1.6, maximum_length = 1.7) # True

True

In [99]:
bond_check(1.6, 0.5, 1.7) # True

True

In [ ]:
bond_check(1.6, 0.5, 1.7) # True

In [100]:
# Adjusting the function now to include the bond check
datafile = open('H2Obondlengths3.txt','w+')

for i in range(len(water_data)):
    for j in range(len(water_data)):
        # this if statement prevents recipricol comparisons
        if i < j:
            # Calling the function calculate_distance()
            D = calculate_distance(atom_coordinates[i], atom_coordinates[j])
            
            # covalent bonds are less than 1.5, don't include bonds greater than 1.5
            # looking for bonds > 0 will cut out the comparisons of an atom to itself
            if bond_check(D) is True:
                print(f'{atom_names[i]} to {atom_names[j]} bond distance: {D: .3f} angstroms.')
                datafile.write(f'{atom_names[i]} to {atom_names[j]} bond distance: {D: .3f} angstroms. \n')

datafile.close()    


O to H1 bond distance:  0.969 angstroms.
O to H2 bond distance:  0.969 angstroms.


In [103]:
# Write a function that reasd in and processes an xy file
# Function name open_xyz
# Input filename
# Two outputs: symbols and coorbinates (return symbols, coordinates)

def open_xyz(filepath):
    """
    Function to open an xyz file, pull the atom names and the x, y, z coordinates
    Input:
        Filename parameter is required and contains the file path for the xyz file
        Assumes that the .xyz file has two header rows, which will be skipped
        Assumes that the data array includes columns in the following order:
            Index 0: atom name
            Index 1: x coordinate
            Index 2: y coordinate
            Index 3: z coordinate
        Coordinates are in Angstroms 
    """
    xyz_file = numpy.genfromtxt(fname = filepath, skip_header = 2, dtype= 'unicode')
    symbols = xyz_file[:, 0] # all rows, column 1 to get name
    coordinates = xyz_file[:, 1:] # all rows, column 2 and on to get numbers
    coordinates = coordinates.astype(numpy.float) # change data type from string to float
    return symbols, coordinates

# Don't put the os.path.join into the function or you have to pass the path into each function. Allows paths to be geenrated in a variety of ways


In [104]:
open_xyz(file_path)

(array(['O', 'H1', 'H2'], dtype='<U9'),
 array([[ 0.      , -0.007156,  0.965491],
        [-0.      ,  0.001486, -0.003471],
        [ 0.      ,  0.931026,  1.207929]]))